<a href="https://colab.research.google.com/github/jopelle/Practica3_Ingenieria_Software/blob/master/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nuestro primer modelo de clasificación

Antes de empezar, pulsa en el botón **Abrir en modo de ensayo** que aparece justo debajo de la barra de herramientas, e inicia sesión con tu cuenta de Google. 

En este [notebook de Jupyter](https://jupyter.org/) vamos a entrenar el algoritmo KNN para clasificar flores de [Iris](https://www.google.com/search?q=flor+de+iris&source=lnms&tbm=isch&sa=X&ved=0ahUKEwi3pvz3goLhAhUFLBoKHbQvDeMQ_AUIDigB&biw=1853&bih=981) en tres posibles clases (Iris setosa, Iris virginica e Iris versicolor), a partir de información como el ancho y largo del sépalo, y el áncho y largo del pétalo.

![alt text](http://latour-marliac.com/323-large_default/iris-versicolor-iris-versicolore.jpg)


## Introducción 

Los notebooks de Jupyter son una manera de interactuar con el lenguaje de programación Python, uno de los lenguajes más utilizados en el ámbito del aprendizaje automático. Estos notebooks, o cuadernos, mezclan celdas de texto con celdas de código. Lo que estás leyendo es una celda de texto en la cual se da una explicación. La siguiente celda, en la que aparece ``1+2``, es una celda de código que se puede ejecutar. Para ello pincha sobre la celda, verás que a la izquierda de la misma aparece un botón con el símbolo de *play*. Haz click sobre dicho botón. Una vez que hagas click la expresión de la celda (``1+2``) se evaluará y se mostrará el resultado. Después de pulsar el botón *play* aparecerá un número donde antes estaba el botón. Esto indica que se ha ejecutado la instrucción.

Las celdas de código se pueden ver como una especie de calculadora donde escribimos el cálculo que queremos realizar, y al pulsar el botón de *play* se ejecuta la instrucción que hayamos introducido. 

In [0]:
1+8

Además de los operadores básicos (como la suma, la resta, la división, etc.) existen muchas funciones para realizar cálculos. Por ejemplo, en la siguiente celda aparece la función ``max`` que calcula el máximo de dos números, el 10 y el 3, que se le pasan como parámetros a la función. Prueba a cambiar los números 10 y 3 por los números que tú quieras, y comprueba que al ejecutar la celda se obtiene el resultado esperado.

In [0]:
max(10,3)

## Librerías necesarias
La siguiente celda carga las librerías de funciones que serán necesarias para construir nuestro modelo. Por así decirlo estamos extendiendo el número de operaciones que se pueden hacer con nuestra calculadora. 

**Es importante que ejecutes (hagas click en el botón play) de todas las celdas de código, y en el orden que aparecen en el cuaderno, de lo contrario es posible que las cosas no funcionen de manera correcta**.

In [0]:
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Entrenando el modelo

Como hemos comentado en las diapositivas, los pasos para entrenar un modelo de clasificación son los siguientes:
1. Obtener un dataset.
2. Extraer descriptores.
3. Partir el dataset.
4. Entrenar un modelo.
5. Evaluar el modelo.

Vamos a ver cada uno de estos pasos.



### 1. Dataset y descriptores

Comenzamos descargando nuestro dataset de muestras de flores de iris y extrayendo los descriptores del mismo. En este caso vamos a descargar directamente el dataset de descriptores. Esto es gracias a que alguien (en concreto, el biólogo [Ronald Fisher](https://en.wikipedia.org/wiki/Ronald_Fisher)) se encargó de recoger una gran cantidad de muestras de flores de iris, de medir la anchura y altura del sépalo y del pétalo de cada una de las muestras, y de clasificarlas en tres posibles categorías: Iris setosa, Iris virginica e Iris versicolor.

Al ejecutar la siguiente celda se descarga un fichero csv con el [dataset de iris](https://en.wikipedia.org/wiki/Iris_flower_data_set). En concreto lo que está haciendo la siguiente celda es descargar de una página web el fichero iris.csv y guardarlo en la nube donde estamos ejecutando este notebook.

In [0]:
!wget https://raw.githubusercontent.com/IA1819/Datasets/master/iris.csv -O iris.csv

--2020-03-01 13:39:18--  https://raw.githubusercontent.com/IA1819/Datasets/master/iris.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4606 (4.5K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   4.50K  --.-KB/s    in 0s      

2020-03-01 13:39:18 (72.1 MB/s) - ‘iris.csv’ saved [4606/4606]



A continuación cargamos el dataset de muestras de plantas de iris en una variable, llamada ``dataset``,  usando la instrucción ``read_csv('iris.csv')`` que lo que hace es leer el fichero que hemos descargado en el paso anterior.

In [0]:
dataset = read_csv('iris.csv')

Al almacenar el dataset en una variable, tenemos una manera sencilla de hacer operaciones sobre el mismo. Por ejemplo, podemos mostrar el valor de esa variable.

In [0]:
dataset

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


El resultado de ejecutar la expresión anterior es una tabla donde cada fila representa una muestra de flor de iris (una *instancia*), y cada columna de esa fila da una propiedad de esa muestra.  Una columna destacada es **class** que nos indica la clase de flor de iris de la muestra.

Como hemos comentado anteriormente, nuestro objetivo es crear un modelo que usando KNN sea capaz de predecir la clase a partir de la información de ancho y alto de sépalo y pétalo (es decir, a partir de los descriptores de la muestra).

En la tabla anterior tenemos juntos los descriptores y la clase de cada muestra, pero los algoritmos de aprendizaje automático necesitan recibirlos por separado. Eso es lo que hacemos al ejecutar la siguiente celda. En concreto nos creamos dos variables llamadas ``descriptores`` y ``clases`` que almacenan, respectivamente, los descriptores y las clase de cada muestra. 

Para ser más concretos, en ``descriptores`` almacenamos la tabla anterior eliminando la última columna, y en ``clases`` almacenamos la última columna de la tabla.

In [0]:
descriptores = dataset.values[:,0:-1]

In [0]:
clases = dataset.values[:,-1]

### 2. Partiendo el dataset.

Ahora vamos a partir el dataset en dos bloques: uno que utilizaremos para entrenar (el *conjunto de entrenamiento*) y otro que utilizaremos para evaluar el rendimiento del modelo que entrenemos (el *conjunto de test*).

Para ello vamos a utilizar la función ``train_test_split`` que recibe los descriptores y las clases y las divide en dos grupos usando un porcentaje. En concreto en este caso vamos a usar el 75% de los datos para entrenar y el 25% para probar, estos son los valores por defecto utilizados por la función ``train_test_split``. También a la función  ``train_test_split`` le pasamos un valor llamado ``random_state`` que sirve para poder reproducir los resultados (de lo contrario cada vez que probáramos este notebook obtendríamos una partición distinta, ya que dicha partición se realiza de manera aleatoria), a este valor se le llama [semilla](https://en.wikipedia.org/wiki/Random_seed).

La instrucción  ``train_test_split`` devuelve 4 valores que son los descriptores del conjunto de entrenamiento, las clases del conjunto de entrenamiento, los descriptores del conjunto de test, y las clases del conjunto de test. Almacenamos dichos valores en las variables correspondientes.

In [0]:
descriptores_entrenamiento, descriptores_test, clases_entrenamiento, clases_test = train_test_split(descriptores,clases,test_size =.1,random_state=42)

### 3. Entrenando el modelo

El siguiente paso consite en entrenar el modelo. Para ello lo primero que vamos a hacer es almacenar en la variable ``knn`` el clasificador KNN. 



In [0]:
knn = KNeighborsClassifier(n_neighbors=1)

En un primer momento este clasificador no sabe nada, y aprenderá cuando lo entrenemos con nuestro dataset. Esta manera de trabajar permite entrenar el mismo clasificador con distintos datasets de manera sencilla.

A continuación mostramos cómo se realiza el entrenamiento, que consite en llamar a la función ``fit`` pasándole como parámetros los descriptores y las clases de entrenamiento. 

In [0]:
knn.fit(descriptores_entrenamiento,clases_entrenamiento)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

Ya hemos entrenado nuestro modelo :-)

Ahora queda probarlo para ver qué tal funciona.

### 4. Evaluando el modelo

A continuación vamos a ver es cómo evaluar el rendimiento del modelo con datos que no ha visto hasta ahora, para ello usaremos el conjunto de test, y calcularemos su *accuracy* (número de aciertos entre número total de predicciones). 

Para ello utilizamos la función ``accuracy_score`` que recibe como parámetro las clases del conjunto de test (es decir el resultado esperado) y las clases predichas por nuestro clasificador (obtenidas mediante la instrucción ``knn.predict(descriptores_test)``).

In [0]:
accuracy_score(clases_test,knn.predict(descriptores_test))

1.0

Hemos obtenido una *accuracy* del 100%. Es decir, que para nuestro conjunto de test, el clasificador que hemos construido acierta siempre. Esto  es algo poco habitual, pero que en este caso, al ser un dataset pequeño y sencillo es posible.   

### 5. Usando el modelo

Una vez que hemos entrenado nuestro modelo y hemos visto que funciona bien en el conjunto de test, podemos pasar a utilizarlo para predecir la clase de nuevas muestras. 

Por ejemplo, si tenemos una nueva muestra con largo de sépalo 5.2, ancho de sépalo 3.3, largo de pétalo 1.6, y ancho de pétalo 0.5; podemos pedirle al modelo que nos diga a la clase que pertenece dicha muestra del siguiente modo:

In [0]:
knn.predict([[5.2,3.3,1.6,0.5],[1.2,0.3,2.6,5.5]])

array(['Iris-setosa', 'Iris-virginica'], dtype=object)

-----
Fin